<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Infeniería de feature

Crear un modelo base para analizar el dataset de partidos<br>
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('partidos.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv > partidos.csv
    else:
        !wget partidos.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("partidos.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
# ¿Cuántos datos quedaron para analizar?
# (filas, columnas)
df.shape

## Fin de la limpieza
No hay datos incompletos o mal ingresados en el dataset, está limpio! No es necesario ningún tipo de limpieza

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
df.head()

Se puede observar que está el puntaje de cada partido pero no el resultado final (wins, loses, draws) --> (ganó, perdió, empató). Agregaremos una columna "results" con el resultado por partido.

In [ ]:

df['result'] = df.apply(lambda x : 'wins' if x.home_score > x.away_score else 'loses' if x.home_score < x.away_score else 'draws', axis=1)
df.head()

In [ ]:
# En el registro de la totalidad de partidos, ¿Cuántos partidos se ganaron, cuántos se pierden y cuántos se empataron?
df['result'].value_counts()

In [ ]:
# Renderizamos el gráfico utilizando el diagrama de cajas
print("¿Cuántos países hay registrados?:", df['home_team'].nunique())

In [ ]:
# Partidos jugados por Argentina de local:
pais_analizar = 'Argentina'
df_arg = df[ df['home_team'] == pais_analizar]
sns.countplot(x='result', data=df_arg).set_title(f"Histórico de {pais_analizar}")
# El countplot nos resuelve no tener que hacer lo siguiente:
#sns.barplot(x=df_arg['result'].value_counts().index, y=df_arg['result'].value_counts())
plt.show()

In [ ]:
# Simplificar el resultado del datset, solo se tendrá encuenta si el local gana (target=1)
# Caso contrario target = 0
df['target'] = df['result'].apply(lambda x: 1 if x == 'wins' else 0)
df

# Entrenar modelo
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle"></div>

In [ ]:
# Crear un numpy array "X" de los features de entrada que se utilizarian
# para entrar al modelo
# Se ha seleccionado los nombres de los equipos rivales, el consejo que
# que para aumentar la performance en vez de ingresar el nombre se realice
# binaryEncoding
# (queda para el almno que desee realizar el proyecto de este dataset)
X = df[['home_team', 'away_team']].values

# Crear un numpy array "y" con la salida/objetivo --> target
y = df['target'].values

In [ ]:
# Crear los dataset de entrenamiento (train) y evaluación (test)
# utilizando la herramienta train_test_split de scikit-learn,
# con la proporción 70%30%
# Ojo! Tener en cuenta que los dataset ahora son numpy array
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Creamos el modelo base
class RandomBaseModel():
  def fit(self,X, y):
    return None

  def predict(self,X):
    return np.random.randint(0, 2, size=X.shape[0])

In [ ]:
# Obtener la salida según el modelo base
random_model = RandomBaseModel()
random_model.fit(X_train, y_train)
y_hat_base = random_model.predict(X_test)

In [ ]:
# Dibujar la matriz de confusión con y_test e y_hat_base
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat_base)
cmd = ConfusionMatrixDisplay(cm, display_labels=['no_wins','wins'])
cmd.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat_base, normalize=True)

In [ ]:
# Calcular el f1_score
from sklearn.metrics import f1_score
f1_score(y_test, y_hat_base)